Preliminary Steps

In [3]:
#Import Pandas and DecisionTree Classifier, 
# Regex, 
# TextBlob, 
# TextFeatures, 
# NLTK,
# and CountVectorizer
import pandas as pd
import re
from sklearn.tree import DecisionTreeClassifier
import nltk
nltk.download("wordnet")
from textblob import Word
import textfeatures as tf
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package wordnet to /home/monty/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/monty/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#Import data as Dataframes, then parse into full dataset with new feature called "ground_truth"
real, fake = pd.read_csv("Data/True.csv"), pd.read_csv("Data/Fake.csv")
real['ground_truth'] = "real"
fake['ground_truth'] = "fake"
dataset = pd.concat([real, fake])

Data Collection and Cleaning

In [5]:
#Extracting amount of capitalized/lowercase words in the title
dataset["titleUpperCaseCount"] = dataset['title'].str.findall(r'[A-Z]').str.len()
dataset["titleLowerCaseCount"] = dataset['title'].str.findall(r'[a-z]').str.len()
#Extracting amount of capitalized/lowercase words in the text
dataset["textUpperCaseCount"] = dataset['text'].str.findall(r'[A-Z]').str.len()
dataset["textLowerCaseCount"] = dataset['text'].str.findall(r'[a-z]').str.len()

In [6]:
#Extract the month and year from the texts
dataset["Year"] = dataset["date"].str.strip(" ").str[-4:]
dataset["Month"]= dataset["date"].str.split(" ").str[0]

In [7]:
#Extract feature counts from words
tf.word_count(dataset,"text", "word_count")
tf.stopwords_count(dataset, "text", "stopword_count")

In [ ]:

dataset['text'] = dataset['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,3), max_features=4000)
counts_sparse = vectorizer.fit_transform(dataset['text'])
counts = pd.DataFrame(counts_sparse.toarray(), index=dataset.index, columns=vectorizer.get_feature_names_out())


ValueError: columns overlap but no suffix specified: Index(['subject', 'date'], dtype='object')

In [ ]:
#Concat dataframes
dataset = pd.concat([dataset, counts], axis=1)

In [ ]:
dataset

,title,text,subject,date,ground_truth,titleUpperCaseCount,titleLowerCaseCount,textUpperCaseCount,textLowerCaseCount,Year,...,you re,you think,you to,you ve,you want,young,your,youtube,zero,zone
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",real,4,48,155,3567,2017,...,0,0,0,0,0,1,0,0,0,0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",real,4,49,119,3217,2017,...,0,0,0,0,0,0,0,0,0,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",real,7,38,117,2134,2017,...,0,0,0,0,0,0,0,0,0,0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",real,8,41,110,1902,2017,...,0,0,0,0,0,0,0,0,0,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",real,4,53,200,3925,2017,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire say As 21WIRE reported earli...,Middle-east,"January 16, 2016",fake,13,38,152,2481,2016,...,0,0,0,0,0,0,0,0,0,0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire say It s a familiar theme. W...,Middle-east,"January 16, 2016",fake,15,46,69,1261,2016,...,0,0,0,0,0,0,0,0,0,0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember w...,Middle-east,"January 15, 2016",fake,12,57,962,19130,2016,...,0,0,0,0,1,0,2,0,1,13
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire say Al Jazeera America will ...,Middle-east,"January 14, 2016",fake,9,42,170,1921,2016,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Split data
holdout = dataset.sample(5000)
dataset = dataset - holdout

NameError: name 'dataset' is not defined

In [ ]:
print(1)

1
